In [2]:
# ! pip install osmnx
import osmnx as ox
import geopandas as gpd
import shapely.geometry
from shapely.geometry import box
import numpy as np
from tqdm import tqdm
import time

def create_subgraphs_from_bboxes_slow(G, bboxes):
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    
    bbox_gdf = gpd.GeoDataFrame({'bbox': bboxes},
                                geometry=[box(west, south, east, north) for west, south, east, north in bboxes],
                                crs=gdf_nodes.crs)

    joined_nodes = gpd.sjoin(gdf_nodes, bbox_gdf, how='left', predicate='within')

    subgraphs = {}
    for bbox in tqdm(bbox_gdf['bbox'], desc="Iterating over bboxes"):
        west, south, east, north = bbox
        nodes_in_bbox = joined_nodes[joined_nodes['bbox'] == bbox].index

        # Filter edges where at least one node is in the bbox
        # Include edge keys for MultiDiGraph
        edges_in_bbox = [(u, v, k) for u, v, k in G.edges(keys=True) if u in nodes_in_bbox or v in nodes_in_bbox]

        # Create subgraph based on these edges
        G_sub = G.edge_subgraph(edges_in_bbox).copy()
        subgraphs[(north, south, east, west)] = G_sub

    return subgraphs


def create_subgraphs_from_bboxes_optimised(G, bboxes):
    gdf_nodes, _ = ox.graph_to_gdfs(G)

    bbox_gdf = gpd.GeoDataFrame({'bbox': bboxes},
                                geometry=[box(west, south, east, north) for west, south, east, north in bboxes],
                                crs=gdf_nodes.crs)

    joined_nodes = gpd.sjoin(gdf_nodes, bbox_gdf, how='left', predicate='within')

    subgraphs = {}
    for bbox in tqdm(bbox_gdf['bbox'], desc="Iterating over bboxes"):
        nodes_in_bbox = joined_nodes[joined_nodes['bbox'] == bbox].index

        # Collect edges by checking neighbors of each node in the bbox
        edges_in_bbox = set()
        for node in nodes_in_bbox:
            for neighbor in G.neighbors(node):
                if G.has_edge(node, neighbor):
                    key = 0 if not G.is_multigraph() else min(G[node][neighbor])
                    edges_in_bbox.add((node, neighbor, key))

        # Create subgraph based on these edges
        G_sub = G.edge_subgraph(edges_in_bbox).copy()
        
        subgraphs[bbox] = G_sub

    return subgraphs

# G_sub = ox.utils_graph.get_largest_component(G_sub, strongly=False)

if __name__ == "__main__":
    # Load the base graph for the entire city
    startime = time.time()
    # G_base = ox.graph_from_place('London', network_type='drive')
    print ("Time taken to get Graph from OSM: ", round(time.time() - startime,2))
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G_base)

    # Determine the bounds of the city
    west, south, east, north = gdf_nodes.unary_union.bounds

    # Calculate the step sizes for latitude and longitude
    N = 50
    lat_step = (north - south) / N
    lon_step = (east - west) / N

    # Generate a grid of bounding boxes (25x25)
    bboxes = []
    for i in range(N):
        for j in range(N):
            bbox_west = west + j * lon_step
            bbox_east = west + (j + 1) * lon_step
            bbox_south = south + i * lat_step
            bbox_north = south + (i + 1) * lat_step
            bboxes.append((bbox_west, bbox_south, bbox_east, bbox_north))

    # Now you can process these bboxes with your function
    # For example:
    subgraphs_slow = create_subgraphs_from_bboxes_slow(G_base, bboxes[1000:60])

    subgraphs_fast = create_subgraphs_from_bboxes_optimised(G_base, bboxes[1000:1500])
    
    # [Add your processing code here]


Time taken to get Graph from OSM:  0.0


Iterating over bboxes: 0it [00:00, ?it/s]
Iterating over bboxes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:04<00:00, 100.63it/s]


In [ ]:
# for i in range(10):
#     print (ox.basic_stats(subgraphs_fast[i]).values())
#     print (ox.basic_stats(subgraphs_slow[i]).values(), ox.basic_stats(subgraphs_slow[i])['m'], len(subgraphs_slow[i].edges))
#     print (" --- \n --- ")


useful_i = []
for i in range(500):
    if len(subgraphs_fast[i].edges)==0 or len(subgraphs_fast[i].nodes)==0 or \
        len(subgraphs_slow[i].edges)==0 or len(subgraphs_slow[i].nodes)==0:
        continue
    useful_i.append(i)
    print (ox.basic_stats(subgraphs_fast[i])['m'], len(subgraphs_fast[i].edges))
    print (ox.basic_stats(subgraphs_slow[i])['m'], len(subgraphs_slow[i].edges))
    print (" --- \n --- ")

In [ ]:
import matplotlib
print(matplotlib.get_backend())
# If necessary, switch to a different backend, e.g., 'TkAgg', 'Qt5Agg', etc.
matplotlib.use('TkAgg')

def plot_separate_subgraphs(subgraphs, title_prefix, num_plots=5):
    for i in useful_i:
        if i in subgraphs:
            G_sub = subgraphs_slow[i]
            fig, ax = plt.subplots(figsize=(6, 6))
            fig.suptitle(f"{title_prefix} - Subgraph {i}")
            ox.plot_graph(G_sub, ax=ax, node_size=20, edge_linewidth=1, node_color='blue')
            ox.plot_graph(subgraphs_fast[i], ax=ax, node_size=20, edge_linewidth=1, node_color='red')
            plt.show()

plot_separate_subgraphs(subgraphs_slow, "Slow Method", num_plots=5)
# plot_separate_subgraphs(subgraphs_fast, "Optimized Method", num_plots=5)


TkAgg


In [6]:
# ! pip install osmnx
import osmnx as ox
import geopandas as gpd
import shapely.geometry
from shapely.geometry import box
import numpy as np
from tqdm import tqdm
import time

bbox_gdf = gpd.GeoDataFrame({'bbox_id': range(len(bboxes))},
                                geometry=[box(west, south, east, north) for west, south, east, north in bboxes],
                                crs=gdf_nodes.crs)


In [10]:
west, south, east, north = bbox_gdf["bbox"][0]

KeyError: 'bbox'

In [1]:
for i in (useful_i):
    print(f"Subgraph slow {i}: {subgraphs_slow[i].number_of_nodes()} nodes, {subgraphs_slow[i].number_of_edges()} edges")
    print(f"Subgraph fast {i}: {subgraphs_fast[i].number_of_nodes()} nodes, {subgraphs_fast[i].number_of_edges()} edges")
    if i >= 5:  # check first few subgraphs
        break

ox.basic_stats(subgraphs_slow[i]).keys()

NameError: name 'useful_i' is not defined

In [6]:
for key in subgraphs_fast:
    print (subgraphs_fast[key].nodes)

[]
[]
[248856498, 248856346, 324253]
[248855428, 3460003462, 21554315, 248855700, 248856346, 1648182303, 1648182304, 1648182305, 1648182306, 1648182309, 21554214, 9320833829, 3927260965, 1648197544, 9501010221, 1648182446, 1648197551, 10807473, 10807474, 21554227, 10807476, 248855860, 11436934834, 248855861, 267519801, 1214809530, 9500980158, 1648182464, 253963073, 253963076, 253963078, 9500980167, 1648182472, 1648182351, 1648182356, 9500980183, 1648182361, 1648182364, 324317, 324316, 324318, 324320, 4471293795, 1648182378, 324331, 1648182381, 1648182384, 9500980211, 9500980212, 9500980213, 9500980216, 445392634, 266527355, 2714945278]
[243512848, 243512852, 1648182303, 245003303, 7428330535, 243512874, 245003307, 2598990378, 243512879, 267519029, 243512886, 267519031, 267519030, 243512890, 16477756, 16477759, 16477760, 243512897, 16718402, 16718401, 16718404, 16718405, 16718406, 243512903, 16477769, 243512906, 16477770, 243512908, 16477773, 243512912, 1648250963, 243512915, 1648250965